## **1. url crawling**

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import logging
import random
import os

# 로그 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

search_query = '기독교'
encoded_query = requests.utils.quote(search_query)

news_urls = set()

base_url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query='

# 고정된 User-Agent 사용
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
}

def get_news_urls(page):
    """각 페이지에서 네이버 뉴스 URL을 수집하는 함수"""
    url = f'{base_url}{encoded_query}&start={page*10}'
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # 더 정확한 뉴스 링크 선택자 사용
        links = soup.select('a.info')

        if not links:
            logging.warning(f'페이지 {page}: 뉴스 기사 없음 (차단 가능성 있음)')
            return False

        for link in links:
            news_url = link['href']
            if 'news.naver.com' in news_url:
                news_urls.add(news_url)

        logging.info(f'페이지 {page}: {len(links)}개 기사 중 {len(news_urls)}개 수집됨')
        return True

    except requests.exceptions.RequestException as e:
        logging.error(f'페이지 {page} 요청 실패: {e}')
        return False

page = 0
max_pages = 200
while len(news_urls) < 1000 and page < max_pages:
    if not get_news_urls(page):
        break
    page += 1
    time.sleep(2)

logging.info(f'총 {len(news_urls)}개의 네이버 뉴스 URL을 수집했습니다.')

save_dir = './url'
os.makedirs(save_dir, exist_ok=True)  # 디렉토토리가 없으면 생성

txt_filename = os.path.join(save_dir, f"{search_query}_naver_news_urls.txt")

with open(txt_filename, 'w', encoding='utf-8') as f:
    for url in news_urls:
        f.write(url + '\n')

logging.info(f"URL 리스트가 {txt_filename} 파일에 저장되었습니다.")


2025-02-16 19:50:03,347 - INFO - 페이지 0: 18개 기사 중 8개 수집됨
2025-02-16 19:50:05,546 - INFO - 페이지 1: 18개 기사 중 16개 수집됨
2025-02-16 19:50:07,721 - INFO - 페이지 2: 16개 기사 중 22개 수집됨
2025-02-16 19:50:09,906 - INFO - 페이지 3: 16개 기사 중 28개 수집됨
2025-02-16 19:50:12,099 - INFO - 페이지 4: 16개 기사 중 34개 수집됨
2025-02-16 19:50:14,275 - INFO - 페이지 5: 14개 기사 중 38개 수집됨
2025-02-16 19:50:16,437 - INFO - 페이지 6: 15개 기사 중 42개 수집됨
2025-02-16 19:50:18,637 - INFO - 페이지 7: 13개 기사 중 45개 수집됨
2025-02-16 19:50:20,802 - INFO - 페이지 8: 15개 기사 중 50개 수집됨
2025-02-16 19:50:22,985 - INFO - 페이지 9: 13개 기사 중 53개 수집됨
2025-02-16 19:50:25,184 - INFO - 페이지 10: 16개 기사 중 54개 수집됨
2025-02-16 19:50:27,354 - INFO - 페이지 11: 15개 기사 중 55개 수집됨
2025-02-16 19:50:29,582 - INFO - 페이지 12: 16개 기사 중 56개 수집됨
2025-02-16 19:50:31,782 - INFO - 페이지 13: 13개 기사 중 59개 수집됨
2025-02-16 19:50:33,951 - INFO - 페이지 14: 14개 기사 중 59개 수집됨
2025-02-16 19:50:36,114 - INFO - 페이지 15: 12개 기사 중 60개 수집됨
2025-02-16 19:50:38,287 - INFO - 페이지 16: 13개 기사 중 60개 수집됨
2025-02-16 19:50:40,479 -

## **2. 본문 Scrapping**

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import time
import logging
import pandas as pd
import re



# 🔹 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 🔹 저장된 URL 파일 불러오기
search_query = "기독교"
txt_filepath = os.path.join("./url", f"{search_query}_naver_news_urls.txt")




# 🔹 뉴스 URL 불러오기
with open(txt_filepath, 'r', encoding='utf-8') as f:
    news_urls = [line.strip() for line in f.readlines()]

logging.info(f"{len(news_urls)}개의 뉴스 URL을 불러왔습니다.")

# 🔹 기사 데이터 크롤링
news_data = []
headers = {'User-Agent': 'Mozilla/5.0'}

for idx, url in enumerate(news_urls):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            logging.warning(f"URL {idx + 1}: 요청 실패 ({response.status_code})")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.select_one('h2#title_area') or soup.select_one('h1')
        content = soup.select_one('#dic_area') or soup.select_one('article') or soup.select_one('.news_end')

        news_data.append([
            url, 
            title.get_text(strip=True) if title else "제목 없음",
            content.get_text(strip=True) if content else "본문 없음"
        ])
        logging.info(f"{idx + 1}: 크롤링 완료 - {title.get_text(strip=True)[:30] if title else '제목 없음'}...")

        time.sleep(1)  # 딜레이 추가

    except Exception as e:
        logging.error(f"URL {idx + 1} 크롤링 실패: {e}")

# 🔹 결과 저장
csv_filename = os.path.join("../Preprocessing", f"{search_query}_content.csv")
pd.DataFrame(news_data, columns=['URL', '제목', '본문']).to_csv(csv_filename, index=False, encoding='utf-8-sig')

logging.info(f"크롤링 완료! 데이터가 '{csv_filename}' 파일에 저장되었습니다.")


2025-02-16 20:06:29,599 - INFO - 763개의 뉴스 URL을 불러왔습니다.
2025-02-16 20:06:29,698 - INFO - 1: 크롤링 완료 - 트럼프 "기독교 공격 중단시킬 것...종교자유위 설치"...
2025-02-16 20:06:30,791 - INFO - 2: 크롤링 완료 - “전한길도 뜬다”…‘尹탄핵 찬반’ 두쪽 나는 광주 금남...
2025-02-16 20:06:31,882 - INFO - 3: 크롤링 완료 - 홍준표 "광주 민주화 상징 금남로서 탄핵 반대 보수단체...
2025-02-16 20:06:32,999 - INFO - 4: 크롤링 완료 - 성경 속 ‘광야’와 같은 탈기독교 사회에서 지녀야 할 ...
2025-02-16 20:06:34,176 - INFO - 5: 크롤링 완료 - 한강에 두 자녀 던져 죽인 20대 아빠 "난 기독교인,...
2025-02-16 20:06:35,342 - INFO - 6: 크롤링 완료 - '전광훈 OUT' 비판 현수막 손배소 … 대법 "초상권...
2025-02-16 20:06:36,500 - INFO - 7: 크롤링 완료 - 교회언론회 이주민 시대 속 “기독교 혐오 경계…무슬림에...
2025-02-16 20:06:37,613 - INFO - 8: 크롤링 완료 - 한국기독교학회 "사람 같은 인공지능, 신학적 인간관 재...
2025-02-16 20:06:38,717 - INFO - 9: 크롤링 완료 - 사병이 자살했다고 군악대가 연주하고 조총까지? [민병래...
2025-02-16 20:06:39,863 - INFO - 10: 크롤링 완료 - [포토뉴스]영월군기독교연합회, 장학금 200만원 기탁...
2025-02-16 20:06:40,948 - INFO - 11: 크롤링 완료 - 김문수 “전광훈은 대한민국 자유민주주의 수호자”...
2025-02-16 20:06:42,033 - INFO - 12: 크롤링 완료 - 계엄 후 최대 인파 몰린 광주… 50m 거리 차벽 두고...